In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [9]:

# Define PyTorch CNN model
class TorchCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        """
        PyTorch CNN 模型
        参数：
        - input_channels: 输入通道数，例如 MNIST 的灰度图像为 1
        - num_classes: 输出类别数，例如 MNIST 为 10 类
        """
        super(TorchCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1),  # 卷积层，输出通道数 32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 最大池化，减少尺寸
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # 卷积层，输出通道数 64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 最大池化
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),  # 全连接层，输出维度 128
            nn.ReLU(),
            nn.Linear(128, num_classes)  # 全连接层，输出维度为类别数
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [10]:
class TorchMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        """
        PyTorch MLP 模型
        参数：
        - input_size: 输入特征维度，例如 MNIST 的 28x28 图像为 784
        - hidden_sizes: 隐藏层的节点数列表，例如 [128, 64]
        - num_classes: 输出类别数，例如 MNIST 为 10 类
        """
        super(TorchMLP, self).__init__()
        layers = []
        in_features = input_size

        # 添加隐藏层
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size

        # 添加输出层
        layers.append(nn.Linear(in_features, num_classes))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 将输入展平为 (batch_size, input_size)
        x = self.model(x)
        return x


In [11]:

# Define sklearn KNN model
def create_knn_model(n_neighbors=5):
    """
    sklearn KNN 模型
    参数：
    - n_neighbors: 邻居数量，默认 5
    """
    return KNeighborsClassifier(n_neighbors=n_neighbors)

# Define sklearn SVM model
def create_svm_model(kernel='linear', C=1.0):
    """
    sklearn SVM 模型
    参数：
    - kernel: 核函数类型，例如 'linear', 'rbf' 等
    - C: 正则化参数，默认为 1.0
    """
    return SVC(kernel=kernel, C=C, probability=True)

In [12]:
def get_model(model_type, **kwargs):
    """
    根据指定类型创建模型
    参数：
    - model_type: 模型类型，支持 'torch_cnn', 'torch_mlp', 'knn', 'svm'
    - kwargs: 传递给具体模型的参数
    """
    if model_type == "torch_cnn":
        input_channels = kwargs.get("input_channels", 1)  # 输入通道数，默认 1
        num_classes = kwargs.get("num_classes", 10)  # 类别数，默认 10
        return TorchCNN(input_channels, num_classes)

    elif model_type == "torch_mlp":  # 处理 torch_mlp 类型
        input_size = kwargs.get("input_size", 784)  # 输入维度，默认 28x28 展平为 784
        hidden_sizes = kwargs.get("hidden_sizes", [128, 64])  # 隐藏层节点数量列表，默认 [128, 64]
        num_classes = kwargs.get("num_classes", 10)  # 类别数，默认 10
        return TorchMLP(input_size, hidden_sizes, num_classes)

    elif model_type == "knn":
        n_neighbors = kwargs.get("n_neighbors", 5)  # KNN 的邻居数量，默认 5
        return create_knn_model(n_neighbors)

    elif model_type == "svm":
        kernel = kwargs.get("kernel", 'linear')  # SVM 的核函数类型，默认 'linear'
        C = kwargs.get("C", 1.0)  # SVM 的正则化参数，默认 1.0
        return create_svm_model(kernel, C)

    else:
        raise ValueError(f"Unsupported model type: {model_type}")


In [13]:
if __name__ == "__main__":
    # Example usage
    torch_model = get_model("torch_cnn", input_channels=1, num_classes=10)
    print(torch_model)

    knn_model = get_model("knn", n_neighbors=3)
    print(knn_model)

    svm_model = get_model("svm", kernel='rbf', C=2.0)
    print(svm_model)
    
    mlp_model = get_model("torch_mlp", input_size=784, hidden_sizes=[128, 64], num_classes=10)
    print(mlp_model)

TorchCNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)
KNeighborsClassifier(n_neighbors=3)
SVC(C=2.0, probability=True)
TorchMLP(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)
